<a href="https://colab.research.google.com/github/gustavo-moretto/WebScrapping_ImoveisCuritiba/blob/main/imoveis_curitiba_v2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [91]:
def analise_imoveis( n_paginas, 
                    comprar_alugar=['aluguel', 'venda'], 
                    tipos_imoveis = ['apartamento_residencial', 'casa_residencial', 'condominio_residencial', 'cobertura_residencial', 'flat_residencial', 'kitnet_residencial', 'sobrado_residencial']):

  import requests
  import pandas as pd
  from bs4 import BeautifulSoup

  import seaborn as sns
  import matplotlib.pyplot as plt
  from time import sleep 

  conjunto_imoveis = {}
  c = 0
  for negocio in comprar_alugar:
    for pagina in range( n_paginas ):
        for tipo_imovel in tipos_imoveis:
          sleep(1)  
          # faz a requisição do site e verifica se esta foi bem-sucedida
          req = requests.get(f"https://www.vivareal.com.br/{negocio}/parana/curitiba/{tipo_imovel}/?pagina={pagina}")

          if req.status_code == 200:
              #print(f'Pag. {pagina}: Requisição bem sucedida!')
              content = req.content
          else:
            print(req.status.code)
            break
            
          # lê o arquivo html com a biblioteca BeautifulSoup
          soup = BeautifulSoup(content, 'html.parser')
            
          # seleciona os dados dos imóveis
          # para isso, encontramos todas as tags <a> com a "class" = "property-card__content-link js-card-title"
          imoveis = soup.find_all('a', attrs={'class':"property-card__content-link js-card-title"})
            
          for imovel in range(len(imoveis)):        
              c += 1
                
              indice_imovel = c
              # extrai a descrição do imóvel
              #descricao = imoveis[imovel].find("span", attrs={"class":"property-card__title js-cardLink js-card-title"}).text                 

              # extrai o endereço e o bairro do imóvel
              endereco = imoveis[imovel].find("span", attrs={"class":"property-card__address"}).text
              end_completo = endereco

              for elemento in range(len(endereco.split())):
                  if endereco.split()[elemento] == '-':
                      end = endereco.split()[elemento + 1]                  

                      # bairros com nomes compostos -> dessa forma, o nome eh extraido da maneira correta                 
                      if end in ['Santa', 'Sitio', 'Cristo', 'Vista', 'Boa', 'Água', 'Cidade', 'Campo', 'Capão']:
                        endereco = endereco.split()[elemento + 1] + ' ' + endereco.split()[elemento + 2]
                        # excluir a virgula do final do bairro
                        endereco = endereco[:-1]                    
                      elif end == 'Alto':
                        endereco = endereco.split()[elemento + 1] + ' ' + endereco.split()[elemento + 2] + ' ' + endereco.split()[elemento + 3]
                        # excluir a virgula do final do bairro
                        endereco = endereco[:-1]
                      
                      # caso nao haja o nome da rua, o bairro sera igual a 'PR'
                      # para extrair o bairro, devemos extrair o primeiro elemento da lista
                      elif end == 'PR':
                        endereco = endereco.split()[0][:-1]
                      else:
                        endereco = endereco.split()[elemento + 1]
                        # excluir a virgula do final do bairro
                        endereco = endereco[:-1]
                      break
              # extrai a área do imóvel (m²)
              # como existem imoveis com a area variavel, p. ex. 20-30, criei uma excecao
              # neste caso, a area do imovel sera igual a 0
              # creio que isso traz mais realidade ao dataset tambem
              try:
                area_m2 = float(imoveis[imovel].find("span", attrs={"class":"property-card__detail-value js-property-card-value property-card__detail-area js-property-card-detail-area"}).text)
              except ValueError:
                area_m2 = 0

              # extrai os itens do imóvel, como mobília, churrasqueira, academia...
              #try:
                  #itens = imoveis[imovel].find("ul", attrs={"class":"property-card__amenities"}).text.split()
              #except AttributeError:
                  #itens = []
              #try:
                  #if itens[-1] == '...':
                      #itens = itens[:-1]
                #except IndexError:
                    #pass

              # número de comodos (caso não possua vaga de garagem, por exemplo, será atribuído valor 0)
              comodos = []
              try:
                  for k in imoveis[imovel].find_all("span", attrs={"class":"property-card__detail-value js-property-card-value"}):
                      comodos.append(float(k.text))
              except ValueError:
                  comodos.append(0)
              if len(comodos) == 3:
                quartos = float(comodos[0])
                banheiros = float(comodos[1])
                vagas = float(comodos[2])
              else:
                quartos = 0
                banheiros = 0
                vagas = 0

              # extrai o valor do aluguel
              try: 
                preco = imoveis[imovel].find("p", attrs={'style':"display: block;"}).text                                                                                         
                try:          
                  if (negocio == 'aluguel' and len(preco.split()[1]) >= 5):            
                    preco = float(preco.split()[1])      
                    preco = preco * 1000                    
                  elif negocio == 'venda' and len(preco.split()) == 2:
                    preco = float(preco.split()[1])
                    preco = preco * 1000                    
                  elif negocio == 'venda' and len(preco.split()) == 6:
                    preco = float(preco.split()[1])
                    preco = preco * 1000                    
                  elif negocio == 'venda' and len(preco.split()) == 5:
                    preco = float(preco.split()[4])
                    preco = preco * 1000                    
                  else:
                    preco = float(preco.split()[1])                    
                    preco = preco * 1000                                                      
                except ValueError:
                  preco = imoveis[imovel].find("p", attrs={'style':"display: block;"}).text
                  preco = preco.replace('.', '')            
                  preco = float(preco.split()[1])
              except AttributeError:                                
                preco = 0                     

              # extrai o valor do condominio (se não existir, será 0)
              # no laco, caso o valor do condominio seja superior a 999.99 reais, sera multiplicado por 1000
              try:
                  condominio = imoveis[imovel].find("strong", attrs={'class':"js-condo-price"}).text
                  if len(condominio.split()[1]) == 3:
                    condominio = float(condominio.split()[1])
                  else:
                    condominio = float(condominio.split()[1])*1000             
              except AttributeError or ValueError:
                  condominio = 0

              conjunto_imoveis[c] = (negocio, tipo_imovel, indice_imovel, end_completo, endereco, area_m2, quartos, banheiros, vagas, preco, condominio)

  df = pd.DataFrame.from_dict(conjunto_imoveis)
  df = df.T
  df = df.rename(columns={ 0:'comprar_alugar', 1:'tipo_imovel', 2:'indice', 3:'end_completo', 4:'bairro', 5:'area_m2', 6:'quartos', 7:'banheiros', 8:'vagas_garagem', 9:'preco', 10:'condominio'})

  colunas_float = ['area_m2', 'quartos', 'banheiros', 'vagas_garagem', 'preco', 'condominio']
  for coluna in colunas_float: 
    df[coluna] = df[coluna].astype( float )
      
  print(f'Base de dados criada: {len(df)} imoveis cadastrados.')
  return df

In [139]:
class database_imoveis():

  def __init__(self, n_paginas):
    self.n_paginas = n_paginas

  def analise_imoveis( self, 
                    comprar_alugar=['aluguel', 'venda'], 
                    tipos_imoveis = ['apartamento_residencial', 'casa_residencial', 'condominio_residencial', 'cobertura_residencial', 'flat_residencial', 'kitnet_residencial', 'sobrado_residencial']):

    import requests
    import pandas as pd
    from time import sleep
    from bs4 import BeautifulSoup

    import seaborn as sns
    import matplotlib.pyplot as plt    

    from google.colab import files
    from datetime import date 

    conjunto_imoveis = {}
    c = 0
    for negocio in comprar_alugar:
      for pagina in range( self.n_paginas ):
          for tipo_imovel in tipos_imoveis:
            sleep(1)  
            # faz a requisição do site e verifica se esta foi bem-sucedida
            req = requests.get(f"https://www.vivareal.com.br/{negocio}/parana/curitiba/{tipo_imovel}/?pagina={pagina}")

            if req.status_code == 200:
                #print(f'Pag. {pagina}: Requisição bem sucedida!')
                content = req.content
            else:
              print(req.status.code)
              break
              
            # lê o arquivo html com a biblioteca BeautifulSoup
            soup = BeautifulSoup(content, 'html.parser')
              
            # seleciona os dados dos imóveis
            # para isso, encontramos todas as tags <a> com a "class" = "property-card__content-link js-card-title"
            imoveis = soup.find_all('a', attrs={'class':"property-card__content-link js-card-title"})
              
            for imovel in range(len(imoveis)):        
                c += 1
                  
                indice_imovel = c
                # extrai a descrição do imóvel
                #descricao = imoveis[imovel].find("span", attrs={"class":"property-card__title js-cardLink js-card-title"}).text                 

                # extrai o endereço e o bairro do imóvel
                endereco = imoveis[imovel].find("span", attrs={"class":"property-card__address"}).text
                end_completo = endereco

                for elemento in range(len(endereco.split())):
                    if endereco.split()[elemento] == '-':
                        end = endereco.split()[elemento + 1]                  

                        # bairros com nomes compostos -> dessa forma, o nome eh extraido da maneira correta                 
                        if end in ['Santa', 'Sitio', 'Cristo', 'Vista', 'Boa', 'Água', 'Cidade', 'Campo', 'Capão']:
                          endereco = endereco.split()[elemento + 1] + ' ' + endereco.split()[elemento + 2]
                          # excluir a virgula do final do bairro
                          endereco = endereco[:-1]                    
                        elif end == 'Alto':
                          endereco = endereco.split()[elemento + 1] + ' ' + endereco.split()[elemento + 2] + ' ' + endereco.split()[elemento + 3]
                          # excluir a virgula do final do bairro
                          endereco = endereco[:-1]
                        
                        # caso nao haja o nome da rua, o bairro sera igual a 'PR'
                        # para extrair o bairro, devemos extrair o primeiro elemento da lista
                        elif end == 'PR':
                          endereco = endereco.split()[0][:-1]
                        else:
                          endereco = endereco.split()[elemento + 1]
                          # excluir a virgula do final do bairro
                          endereco = endereco[:-1]
                        break
                # extrai a área do imóvel (m²)
                # como existem imoveis com a area variavel, p. ex. 20-30, criei uma excecao
                # neste caso, a area do imovel sera igual a 0
                # creio que isso traz mais realidade ao dataset tambem
                try:
                  area_m2 = float(imoveis[imovel].find("span", attrs={"class":"property-card__detail-value js-property-card-value property-card__detail-area js-property-card-detail-area"}).text)
                except ValueError:
                  area_m2 = 0

                # extrai os itens do imóvel, como mobília, churrasqueira, academia...
                #try:
                    #itens = imoveis[imovel].find("ul", attrs={"class":"property-card__amenities"}).text.split()
                #except AttributeError:
                    #itens = []
                #try:
                    #if itens[-1] == '...':
                        #itens = itens[:-1]
                  #except IndexError:
                      #pass

                # número de comodos (caso não possua vaga de garagem, por exemplo, será atribuído valor 0)
                comodos = []
                try:
                    for k in imoveis[imovel].find_all("span", attrs={"class":"property-card__detail-value js-property-card-value"}):
                        comodos.append(float(k.text))
                except ValueError:
                    comodos.append(0)
                if len(comodos) == 3:
                  quartos = float(comodos[0])
                  banheiros = float(comodos[1])
                  vagas = float(comodos[2])
                else:
                  quartos = 0
                  banheiros = 0
                  vagas = 0

                # extrai o valor do aluguel
                try: 
                  preco = imoveis[imovel].find("p", attrs={'style':"display: block;"}).text                                                                                         
                  try:          
                    if (negocio == 'aluguel' and len(preco.split()[1]) >= 5):            
                      preco = float(preco.split()[1])      
                      preco = preco * 1000                    
                    elif negocio == 'venda' and len(preco.split()) == 2:
                      preco = float(preco.split()[1])
                      preco = preco * 1000                    
                    elif negocio == 'venda' and len(preco.split()) == 6:
                      preco = float(preco.split()[1])
                      preco = preco * 1000                    
                    elif negocio == 'venda' and len(preco.split()) == 5:
                      preco = float(preco.split()[4])
                      preco = preco * 1000                    
                    else:
                      preco = float(preco.split()[1])                    
                      preco = preco * 1000                                                      
                  except ValueError:
                    preco = imoveis[imovel].find("p", attrs={'style':"display: block;"}).text
                    preco = preco.replace('.', '')            
                    preco = float(preco.split()[1])
                except AttributeError:                                
                  preco = 0                     

                # extrai o valor do condominio (se não existir, será 0)
                # no laco, caso o valor do condominio seja superior a 999.99 reais, sera multiplicado por 1000
                try:
                    condominio = imoveis[imovel].find("strong", attrs={'class':"js-condo-price"}).text
                    if len(condominio.split()[1]) == 3:
                      condominio = float(condominio.split()[1])
                    else:
                      condominio = float(condominio.split()[1])*1000             
                except AttributeError or ValueError:
                    condominio = 0

                conjunto_imoveis[c] = (negocio, tipo_imovel, indice_imovel, end_completo, endereco, area_m2, quartos, banheiros, vagas, preco, condominio)

    self.df = pd.DataFrame.from_dict(conjunto_imoveis)
    self.df = self.df.T
    self.df = self.df.rename(columns={ 0:'comprar_alugar', 1:'tipo_imovel', 2:'indice', 3:'end_completo', 4:'bairro', 5:'area_m2', 6:'quartos', 7:'banheiros', 8:'vagas_garagem', 9:'preco', 10:'condominio'})

    colunas_float = ['area_m2', 'quartos', 'banheiros', 'vagas_garagem', 'preco', 'condominio']
    for coluna in colunas_float: 
      self.df[coluna] = self.df[coluna].astype( float )
        
    print(f'Base de dados criada: {len(self.df)} imoveis cadastrados.')

    # download da base de dados
    download_basedados = str(input('Deseja fazer o download da base de dados [s/n]: '))
    if download_basedados.upper() == 'S':
      today = date.today()
      print(f'Download do arquivo df_imoveis_{today}.csv')
      self.df.to_csv(f'df_imoveis_{today}.csv')
      files.download(f'df_imoveis_{today}.csv')
    else:
      print('Base de dados nao foi baixada.')
      
    return self.df

In [140]:
database = database_imoveis(20)
df = database.analise_imoveis()
df.head()

Base de dados criada: 9820 imoveis cadastrados.
Deseja fazer o download da base de dados [s/n]: s
Download do arquivo df_imoveis_2022-06-17.csv


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

,comprar_alugar,tipo_imovel,indice,end_completo,bairro,area_m2,quartos,banheiros,vagas_garagem,preco,condominio
1,aluguel,apartamento_residencial,1,"Rua Manoel Eufrásio, 1231 - Juvevê, Curitiba - PR",Juvevê,40.0,1.0,1.0,1.0,2600.0,550.0
2,aluguel,apartamento_residencial,2,"Rua Castro Alves, 75 - Batel, Curitiba - PR",Batel,224.0,4.0,4.0,3.0,3900.0,1000.0
3,aluguel,apartamento_residencial,3,"Rua Silveira Peixoto, 613 - Água Verde, Curiti...",Água Verde,68.0,2.0,1.0,0.0,1300.0,0.0
4,aluguel,apartamento_residencial,4,"Rua Tijucas do Sul, 2680 - Sitio Cercado, Curi...",Sitio Cercado,60.0,2.0,1.0,1.0,1100.0,300.0
5,aluguel,apartamento_residencial,5,"Alameda Júlia da Costa, 911 - Bigorrilho, Curi...",Bigorrilho,25.0,1.0,1.0,0.0,1790.0,350.0


# Imports

In [163]:
import pandas as pd
pd.set_option('display.float_format', lambda x: '%.5f' % x)
import numpy as np

import seaborn as sns
import matplotlib.pyplot as plt

In [143]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 9820 entries, 1 to 9820
Data columns (total 11 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   comprar_alugar  9820 non-null   object 
 1   tipo_imovel     9820 non-null   object 
 2   indice          9820 non-null   object 
 3   end_completo    9820 non-null   object 
 4   bairro          9820 non-null   object 
 5   area_m2         9820 non-null   float64
 6   quartos         9820 non-null   float64
 7   banheiros       9820 non-null   float64
 8   vagas_garagem   9820 non-null   float64
 9   preco           9820 non-null   float64
 10  condominio      9820 non-null   float64
dtypes: float64(6), object(5)
memory usage: 920.6+ KB


In [170]:
# criarei uma coluna de vendas e uma coluna de aluguel que receberao os valores 0 caso a afirmativa seja falsa
# e recebera 1 caso a afirmativa seja verdadeira
df['venda'] = df['comprar_alugar'].apply( lambda x: 1 if x == 'venda' else 0)
df['aluguel'] = df['comprar_alugar'].apply( lambda x: 1 if x == 'aluguel' else 0)

# Analise Descritiva de Dados

In [142]:
df.describe(include='all')

,comprar_alugar,tipo_imovel,indice,end_completo,bairro,area_m2,quartos,banheiros,vagas_garagem,preco,condominio
count,9820,9820,9820.0,9820,9820,9820.000000,9820.000000,9820.000000,9820.000000,9.820000e+03,9820.000000
unique,2,7,9820.0,545,76,NaN,NaN,NaN,NaN,NaN,NaN
top,venda,apartamento_residencial,1.0,"Santa Felicidade, Curitiba - PR",Centro,NaN,NaN,NaN,NaN,NaN,NaN
freq,5040,1440,1.0,101,1494,NaN,NaN,NaN,NaN,NaN,NaN
mean,NaN,NaN,NaN,NaN,NaN,144.368941,2.517006,2.486762,1.837169,4.883877e+05,1598.408758
std,NaN,NaN,NaN,NaN,NaN,140.988551,1.298826,1.544178,1.771400,8.193856e+05,9280.755901
min,NaN,NaN,NaN,NaN,NaN,0.000000,0.000000,0.000000,0.000000,0.000000e+00,0.000000
25%,NaN,NaN,NaN,NaN,NaN,42.000000,1.000000,1.000000,1.000000,3.500000e+03,0.000000
50%,NaN,NaN,NaN,NaN,NaN,109.000000,3.000000,2.000000,2.000000,2.000000e+05,180.000000
75%,NaN,NaN,NaN,NaN,NaN,199.000000,3.000000,3.000000,2.000000,6.800000e+05,600.000000


In [169]:
df.head()

,comprar_alugar,tipo_imovel,indice,end_completo,bairro,area_m2,quartos,banheiros,vagas_garagem,preco,condominio
1,aluguel,apartamento_residencial,1,"Rua Manoel Eufrásio, 1231 - Juvevê, Curitiba - PR",Juvevê,40.00000,1.00000,1.00000,1.00000,2600.00000,550.00000
2,aluguel,apartamento_residencial,2,"Rua Castro Alves, 75 - Batel, Curitiba - PR",Batel,224.00000,4.00000,4.00000,3.00000,3900.00000,1000.00000
3,aluguel,apartamento_residencial,3,"Rua Silveira Peixoto, 613 - Água Verde, Curiti...",Água Verde,68.00000,2.00000,1.00000,0.00000,1300.00000,0.00000
4,aluguel,apartamento_residencial,4,"Rua Tijucas do Sul, 2680 - Sitio Cercado, Curi...",Sitio Cercado,60.00000,2.00000,1.00000,1.00000,1100.00000,300.00000
5,aluguel,apartamento_residencial,5,"Alameda Júlia da Costa, 911 - Bigorrilho, Curi...",Bigorrilho,25.00000,1.00000,1.00000,0.00000,1790.00000,350.00000


In [146]:
# Imoveis p/ vender vs Imoveis p/ alugar

print('Imoveis a Venda vs Imoveis p/ Alugar')
df[['comprar_alugar', 'tipo_imovel']].groupby('comprar_alugar').count()

Imoveis a Venda vs Imoveis p/ Alugar


,tipo_imovel
comprar_alugar,
aluguel,4780
venda,5040


In [148]:
# Numero de imoveis por tipo

print('Numero de imoveis por tipo')
df[['comprar_alugar', 'tipo_imovel']].groupby('tipo_imovel').count()

Numero de imoveis por tipo


,comprar_alugar
tipo_imovel,
apartamento_residencial,1440
casa_residencial,1440
cobertura_residencial,1440
condominio_residencial,1440
flat_residencial,1180
kitnet_residencial,1440
sobrado_residencial,1440


In [164]:
df[df['comprar_alugar'] == 'venda'].groupby('tipo_imovel').agg({'preco': ['mean', 'min', 'max', 'std']})

preco                                         
                                 mean          min           max           std
tipo_imovel                                                                   
apartamento_residencial  681871.43472 125000.00000 3799000.00000  658310.77867
casa_residencial        1234159.58333   3200.00000 8200000.00000 1457120.41677
cobertura_residencial   1274559.41667   2500.00000 7200000.00000  927046.79730
condominio_residencial  1583123.86667      0.00000 6700000.00000 1261112.31559
flat_residencial         283447.22222 115000.00000  450000.00000   89560.08833
kitnet_residencial       217001.36389 109000.00000  451200.00000   81778.70134
sobrado_residencial      740869.97222 120000.00000 1520000.00000  314158.28845